从Figma标注SVG中提取控制点坐标和节点bbox

标注规则：
1. 所有控制点和bbox均在一个group下，控制点使用circle，bbox使用rect
2. 控制点index从1开始递增
3. bbox的id为该node的名称，每个id需要不同

In [12]:
import xml.etree.ElementTree as ET
from svgelements import Rect, Matrix
from unicode_converter import extract_and_decode_ids
import json, os, imagesize

out_dir = "../../src/assets/sishi"
backgroundId = 'background'

original_size = imagesize.get(f"{out_dir}/img.jpeg")
background_size = [0, 0]

svg_file = "2-temp.svg"
chinsese_svg = extract_and_decode_ids(svg_file)
if chinsese_svg:
    with open(svg_file, 'w', encoding='utf-8') as file:
        file.write(chinsese_svg)


# 解析SVG文件
tree = ET.parse(svg_file)
root = tree.getroot()
# 命名空间 (一般SVG文件会包含默认的命名空间)
# 根据具体的SVG文件，可能需要调整命名空间前缀
namespace = {'svg': 'http://www.w3.org/2000/svg'}

# 查找所有一级 <g> 元素
background = root.find(f".//svg:rect[@id='{backgroundId}']", namespace)  # 使用命名空间避免解析问题
background_size = [float(background.attrib['width']), float(background.attrib['height'])] 
ratio = original_size[0] / background_size[0]

rects = root.findall('.//svg:rect', namespace)  # 使用命名空间避免解析问题
circles = root.findall('.//svg:circle', namespace)  # 使用命名空间避免解析问题


rect_dict = {}
circle_dict = {}
for circle in circles:
    attr = circle.attrib
    circle_dict[attr['id']] = [float(attr['cx']) * ratio, float(attr['cy']) * ratio]

for rect in rects:
    attr = rect.attrib
    rect_pos = [[float(attr['x']) * ratio, float(attr['y']) * ratio], [float(attr['width']) * ratio, float(attr['height']) * ratio]] # 以rect中心为相对坐标 (大小为标注svg大小)
    # 乘以ratio换算为根据原图大小
    rect_dict[attr['id']] = {
        "name": attr['id'],
        "controlPoints": [],
        "position": rect_pos,
    }


with open("./link.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(circle_dict, ensure_ascii=False))


获取树的结构，包括逻辑上的父子关系和布局上的父子节点之间连线的控制点

手动保存在struct.json中，每个节点的名称名称规范: ```{name}_{controlPointId}_{controlPointId}_{controlPointId}```

其中name为名称，之后使用下划线分割该节点到其父节点之间控制点的id

```js
{
    A_:{
        B_ctp1_ctp2 : {
            D_ctp3_ctp4: {},
        },
        C_ctp1_ctp2_ctp2 : {}
    }
}
```


In [13]:
import json, os

with open("./2-struct.json", "r", encoding="utf-8") as f:
    struct = json.load(f)

def convert_custom_format_to_standard_tree(custom_data):
    """
    将自定义的嵌套字典格式转换为标准的 "name"/"children" 树形结构。

    Args:
        custom_data (dict): 输入的嵌套字典，例如 {'A': {'B': {}, 'C': {'D': {}}}}

    Returns:
        dict: 标准的树形结构字典，例如 {"name": "A", "children": [...]}
              如果输入只有一个根节点，则返回该根节点字典。
              如果输入有多个顶级节点（虽然你的例子只有一个），理论上可以返回一个列表。
              这里我们假设输入字典的顶层代表一个或多个根节点。
    """

    # 内部递归函数，处理一个级别的子节点
    def process_children(children_dict):
        nodes_list = []
        if not isinstance(children_dict, dict):
            return [] # 如果不是字典，则没有子节点

        for name, grandchildren_dict in children_dict.items():
            _name = name.split('_')[0]
            _controlPoints = name.split('_')[1:]
            node = rect_dict[_name]
            node['controlPoints'] = [int(i) for i in _controlPoints]
            node["children"] = []
            # 递归处理孙子节点
            if isinstance(grandchildren_dict, dict) and grandchildren_dict: # 检查是否是非空字典
                sub_children = process_children(grandchildren_dict)
                if sub_children: # 仅当确实有子节点时才添加 'children' 键
                    node["children"] = sub_children
            # 如果 grandchildren_dict 是空字典 {}，则当前 node 是叶子节点，不添加 "children"
            nodes_list.append(node)
        return nodes_list

    # 处理顶层节点
    # 你的输入 {'汤': {...}} 只有一个顶层键
    if not isinstance(custom_data, dict) or not custom_data:
        return None # 或返回空列表/字典，或抛出错误

    # 假设输入字典的第一个键是树的根
    # 如果可能有多个顶级节点，你需要遍历 custom_data.items()
    # 但根据你的例子，只有一个根 "汤"
    
    root_name = list(custom_data.keys())[0] # 获取第一个键作为根节点名
    root_children_data = custom_data[root_name] # 获取根节点的子节点数据

    output_tree_root = rect_dict[root_name]
    
    if isinstance(root_children_data, dict) and root_children_data:
        children_nodes = process_children(root_children_data)
        if children_nodes:
            output_tree_root["children"] = children_nodes
            
    return output_tree_root


nodeJsonData = convert_custom_format_to_standard_tree(struct)
with open("./node.json", "w", encoding="utf-8") as f:
    f.write(json.dumps(nodeJsonData, ensure_ascii=False))